# Problem Statement :- CCD

Create a model which can extract quantity and item name from any sentence

Like “ i want 1 cappuccino and 2 flat white from CCD”

Here item:quantity = 1:cappuccino and 2: flat white 
Store name = CCD

We can train the menu of Café Coffee Day from ccdonline.in. This model should autocorrect item name too e.g capucino = cappuccino 

In [ ]:
### Install the NLTK
!pip install nltk

In [ ]:
## import packages
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.chunk import ne_chunk
nltk.download('maxent_ne_chunker')
nltk.download('words')
from nltk.corpus import words
from nltk.metrics.distance import jaccard_distance
from nltk.util import ngrams
from nltk.metrics.distance  import edit_distance
from nltk import Tree

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/apple/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /Users/apple/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [ ]:
sentence = 'i want 1 capucino and 2 flat white from CCD'

In [ ]:
## Checking details of POS
ne_tree = ne_chunk(pos_tag(word_tokenize(sentence)))
print(ne_tree)

(S
  i/NN
  want/VBP
  1/CD
  capucino/NN
  and/CC
  2/CD
  flat/JJ
  white/JJ
  from/IN
  (ORGANIZATION CCD/NN))


In [ ]:
 ## Work Tokenizing and Pos   
def preprocess(sent):
    sent = nltk.word_tokenize(sent)
    sent = nltk.pos_tag(sent)
    return sent

In [ ]:
sent = preprocess(sentence)
sent

[('i', 'NN'),
 ('want', 'VBP'),
 ('1', 'CD'),
 ('capucino', 'NN'),
 ('and', 'CC'),
 ('2', 'CD'),
 ('flat', 'JJ'),
 ('white', 'JJ'),
 ('from', 'IN'),
 ('CCD', 'NN')]

In [ ]:
pattern = 'NP: {<CD><JJ>*<NN>*}'

In [ ]:
# parse with regex pattern
cp = nltk.RegexpParser(pattern)
cs = cp.parse(sent)
print(cs,type(cs))

(S
  i/NN
  want/VBP
  (NP 1/CD capucino/NN)
  and/CC
  (NP 2/CD flat/JJ white/JJ)
  from/IN
  CCD/NN) <class 'nltk.tree.Tree'>


In [ ]:
list(cs)

[('i', 'NN'),
 ('want', 'VBP'),
 Tree('NP', [('1', 'CD'), ('capucino', 'NN')]),
 ('and', 'CC'),
 Tree('NP', [('2', 'CD'), ('flat', 'JJ'), ('white', 'JJ')]),
 ('from', 'IN'),
 ('CCD', 'NN')]

In [ ]:
for i in list(cs):
    print(type(i))

<class 'tuple'>
<class 'tuple'>
<class 'nltk.tree.Tree'>
<class 'tuple'>
<class 'nltk.tree.Tree'>
<class 'tuple'>
<class 'tuple'>


In [ ]:
all_item = []
store_name = []
for i in list(cs):
    if type(i) == Tree:
        all_item.append(i)
    else:
        store_name.append(i)

In [ ]:
all_item

[Tree('NP', [('1', 'CD'), ('capucino', 'NN')]),
 Tree('NP', [('2', 'CD'), ('flat', 'JJ'), ('white', 'JJ')])]

In [ ]:
store_name[-1][0]

'CCD'

In [ ]:
all_items_cln = []
for i in all_item:
    tmp = ""
    for j in list(i):
        tmp = tmp + " " + j[0]
    all_items_cln.append(tmp)
print(all_items_cln)

[' 1 capucino', ' 2 flat white']


In [ ]:
#smaple list of beverage from ccdonline.in
correct_spellings = ["cappuccino","flat white","regular cappuccino","frappes","darjeeling tea","dark chocolate"]

#this section will correct the beverage name
def correct_item_name(item_list):
    for i in range(len(item_list)):
        entries= [item_list[1]]
        for entry in entries:
            temp = [(edit_distance(entry, w),w) for w in correct_spellings if w[0]==entry[0]]
            #store the correct beverage as variable
            corr_item_name = (sorted(temp, key = lambda val:val[0])[0][1])
        entries = corr_item_name
        return entries

store_qty_item = []
nested_dict_list ={}
nested_dict_list['Store name'] = {}
nested_dict_list['Store name'] = store_name[-1][0].upper()
for i in all_items_cln:
    item_list = i.lstrip().split(' ',1)
    item_list[1] = correct_item_name(item_list)
    nested_dict_list['Item'] = item_list[1]
    nested_dict_list['Quantity'] = item_list[0]
    print(nested_dict_list )   

{'Store name': 'CCD', 'Item': 'cappuccino', 'Quantity': '1'}
{'Store name': 'CCD', 'Item': 'flat white', 'Quantity': '2'}
